In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [2]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocess the data
## Drop the irrelevant features
df = df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [4]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
## Encode the categorical variables
label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])

In [6]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df["Geography"].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [8]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder_geo = OneHotEncoder(sparse_output=False)
geo_encoder = onehotencoder_geo.fit_transform(df[["Geography"]])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
onehotencoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_encoded_df = pd.DataFrame(data=geo_encoder,columns=onehotencoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
df = pd.concat([df,geo_encoded_df],axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,France,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
df.drop(["Geography"],axis=1,inplace=True)

In [13]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
pickle.dump(label_encoder_gender,open("label_encoder_gender.pkl","wb"))
pickle.dump(onehotencoder_geo,open("one_hot_encoder_geo.pkl","wb"))

In [15]:
## Train_Test_splXt
X = df.drop(columns=["Exited"],axis=1)
y = df["Exited"]

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [18]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [20]:
X_test

array([[-0.57749609,  0.91324755, -0.6557859 , ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.29729735,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.52560743, -1.09499335,  0.48508334, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.81311987, -1.09499335,  0.77030065, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.41876609,  0.91324755, -0.94100321, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.24540869,  0.91324755,  0.00972116, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [21]:
pickle.dump(scaler,open("scaler.pkl","wb"))

### ANN Training

In [22]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime


In [23]:
## Build our ANN model
model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)), ## HL1 connected with input layer
    Dense(32,activation="relu"), ## HL2
    Dense(1,activation="sigmoid")
])


In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

In [29]:
## Compile the model - To perform forward and backward propagation
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=["accuracy"])


In [30]:
## Setup the tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [31]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
tensorflow_callbacks = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [34]:
## Setup early stopping
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [35]:
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callbacks,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3367 - accuracy: 0.8612 - val_loss: 0.3463 - val_accuracy: 0.8605
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3327 - accuracy: 0.8649 - val_loss: 0.3459 - val_accuracy: 0.8615
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3328 - accuracy: 0.8660 - val_loss: 0.3541 - val_accuracy: 0.8580
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3284 - accuracy: 0.8664 - val_loss: 0.3557 - val_accuracy: 0.8565
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3310 - accuracy: 0.8645 - val_loss: 0.3365 - val_accuracy: 0.8690
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3251 - accuracy: 0.8651 - val_loss: 0.3400 - val_accuracy: 0.8620
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3246 - accuracy: 0.8662 - val_loss: 0.3666 - val_accuracy: 0.8580

In [36]:
model.save("model.h5")

c:\Users\venka\OneDrive\Desktop\END-TO-END_DEEP_LRNG\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
## Load Tensorboard Extension
%load_ext tensorboard

In [39]:
%tensorboard --logdir logs/fit 

Reusing TensorBoard on port 6006 (pid 38396), started 0:03:00 ago. (Use '!kill 38396' to kill it.)